In [2]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam import WebAPI
import time
import os
import glob
import json
import tensorflow as tf
print(tf.__version__)
import sys
from IPython import display
import requests
#dont keep api key in github!!!
api_file = os.path.join('..','..','..','apikeys','steam_api_key.txt')
with open(api_file, 'r') as fin:
    api = WebAPI(key=fin.readline())

2.1.0


In [3]:
#declare interface urls
#dota2_beta should be used for testing in order to not aggressively make API requests
dota2_id = '570'
dota2_beta_id = '205790'

hero_list = api.call('IEconDOTA2_'+dota2_id+'.GetHeroes')
hero_vocab=[int(hero_id['id']) for hero_id in hero_list['result']['heroes']]
print(hero_vocab)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 31, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 109, 110, 111, 105, 112, 113, 108, 114, 120, 119, 121, 129, 126, 128]


# Build TFRecords file
Puts all training data in the training_data folder into one TFRecords file
Do this to nicely handle the features that are numpy arrays (radiant_heroes and dire_heroes)

In [25]:
dota2_df = None
files=glob.glob(os.path.join('Dota_data','mixed_skill','*.csv'))
files.sort(key=os.path.getmtime)
print(files[0])
for filename in files[-2:-1]:
    if type(dota2_df)==None:
        dota2_df = pd.read_csv(filename, usecols=range(0,27))
    else:
        dota2_df = pd.concat([dota2_df, pd.read_csv(filename, usecols=range(0,27))],ignore_index=True)

dota2_df = dota2_df.reindex(np.random.permutation(dota2_df.index))
print(dota2_df.columns)
dota2_df.describe()

Dota_data/mixed_skill/20191111API_4725403110-4781085451.csv
Index(['Unnamed: 0', 'match_id', 'start_time', 'duration', 'game_mode',
       'lobby_type', 'radiant_win', 'radiant_kills', 'dire_kills',
       'radiant_gold', 'dire_gold', 'radiant_xp', 'dire_xp',
       'radiant_tower_status', 'dire_tower_status', 'first_blood_time',
       'radiant_hero0', 'dire_hero0', 'radiant_hero1', 'dire_hero1',
       'radiant_hero2', 'dire_hero2', 'radiant_hero3', 'dire_hero3',
       'radiant_hero4', 'dire_hero4', 'skill_level'],
      dtype='object')


,Unnamed: 0,match_id,start_time,duration,game_mode,lobby_type,radiant_win,radiant_kills,dire_kills,radiant_gold,...,radiant_hero0,dire_hero0,radiant_hero1,dire_hero1,radiant_hero2,dire_hero2,radiant_hero3,dire_hero3,radiant_hero4,dire_hero4
count,287747.000000,2.877470e+05,2.877470e+05,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,...,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000,287747.000000
mean,143873.000000,4.757205e+09,1.558035e+09,1883.274376,20.427487,3.564892,0.547387,31.412185,30.583144,82673.295678,...,52.836492,52.891891,52.989744,53.108303,52.965915,53.092828,52.908607,53.007576,52.778799,52.953768
std,83065.548292,1.988639e+07,5.790148e+05,688.588706,5.608296,3.499404,0.497750,13.370800,14.402033,32662.411628,...,35.703343,35.686194,35.790254,35.749790,35.745712,35.752778,35.753868,35.744160,35.769053,35.691874
min,0.000000,4.725403e+09,1.557130e+09,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71936.500000,4.733830e+09,1.557367e+09,1374.000000,22.000000,0.000000,0.000000,22.000000,20.000000,60838.725000,...,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,143873.000000,4.749397e+09,1.557800e+09,1874.000000,22.000000,7.000000,1.000000,31.000000,31.000000,80357.333333,...,48.000000,47.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,47.000000,48.000000
75%,215809.500000,4.775588e+09,1.558573e+09,2343.000000,23.000000,7.000000,1.000000,41.000000,41.000000,101837.516667,...,83.000000,83.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
max,287746.000000,4.781085e+09,1.558752e+09,7123.000000,23.000000,7.000000,1.000000,115.000000,167.000000,444829.066667,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


In [5]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [6]:
def hero_to_hero_indices(heroes):
    hero_indices=[]
    for hero in heroes:
        hero_indices.append(hero_vocab.index(hero))
    return hero_indices

In [7]:
def convert_to_tfrecords(filename, examples, targets, target_name):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets[target_name])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the arrays
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        target = target_array[i]
        
        # Create a feature
        if target_name=='radiant_win':
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'label': _int64_feature(target)}
        else:
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'label': _float_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [57]:
def convert_to_tfrecords_indivheroes(filename, examples, targets):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is already reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    rad_hero_list=[None]*5;dire_hero_list=[None]*5;
    #load columns into 2D lists
    for i in range(5):
        rad_hero_list[i]=np.array(examples['radiant_hero'+str(i)])
        dire_hero_list[i]=np.array(examples['dire_hero'+str(i)])
        
    #skill_level_list=np.array(examples['skill_level'])
    target_list = np.array(targets['radiant_win'])
    
    for i in range(len(rad_hero_list[0])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Create a feature
        feature = {}
        for j in range(5):
            rad_hero=rad_hero_list[j][i]
            dire_hero=dire_hero_list[j][i]
            feature['radiant_hero'+str(j)]= _int64_feature(rad_hero)
            feature['dire_hero'+str(j)]=_int64_feature(dire_hero)
            
        #feature['skill_level']= _int64_feature(skill_level_list[i])
        feature['target']= _int64_feature(target_list[i])

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [29]:
def convert_to_tfrecords_novocab(filename, examples, targets, target_name):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets[target_name])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the arrays
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        rad_heroes_index=hero_to_hero_indices(rad_heroes)
        dire_heroes_index=hero_to_hero_indices(dire_heroes)

        target = target_array[i]
        
        # Create a feature
        if target_name=='radiant_win':
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'label': _int64_feature(target)}
        else:
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'label': _float_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [10]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [11]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        rad_heroes_index=hero_to_hero_indices(rad_heroes)
        dire_heroes_index=hero_to_hero_indices(dire_heroes)
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [12]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_triple_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        rad_heroes_index=hero_to_hero_indices(rad_heroes)
        dire_heroes_index=hero_to_hero_indices(dire_heroes)
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [13]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_penta_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    towdam_diff_array = np.array(target_df['towdam_diff'])
    total_towdam_array = np.array(target_df['total_towdam'])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        rad_heroes_index=hero_to_hero_indices(rad_heroes)
        dire_heroes_index=hero_to_hero_indices(dire_heroes)
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        towdam_diff = towdam_diff_array[i]
        total_towdam = total_towdam_array[i]
    
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold),
                   'tower_damage_diff': _float_feature(towdam_diff),
                   'total_tower_damage': _float_feature(total_towdam)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [14]:
def convert_to_tfrecords_embedding(filename, examples, targets):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    same_side_hero_array = np.array(examples['same_side_heroes'])
    opp_side_hero_array = np.array(examples['opp_side_heroes'])
    target_array = np.array(targets['hero_to_predict'])
    print(len(target_array))
    for i in range(len(same_side_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the arrays
        same_side_hero = same_side_hero_array[:][i]
        opp_side_hero = opp_side_hero_array[:][i]
        
        #convert from hero id to just hero index (from 0-len(vocab))
        same_side_heroes_index=hero_to_hero_indices(same_side_hero)
        opp_side_heroes_index=hero_to_hero_indices(opp_side_hero)

        target = target_array[i]
        
        # Create a feature
        feature = {'same_side_heroes': _int64_list_feature(same_side_heroes_index),
                   'opp_side_heroes': _int64_list_feature(opp_side_heroes_index),
                   'label': _int64_feature(target)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()

In [15]:
#function to take a single random hero from an example for each hero for each example
def feature_target_hero_split(dota2_df):
    """Take dota2_df and create a dataframe containing same_side and opp_side
        heroes with a single hero alone to be predicted.
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                feature_df: pandas DataFrame containing only feature columns
                target_df: pandas DataFrame containing only the hero target
    """  
    feature_df = pd.DataFrame()
    target_df = pd.DataFrame()

    hero_to_predict_list=[]
    same_side_heroes=[]
    opp_side_heroes=[]
    #split heroes into a hero to predict and opp_side and same_side 
    #    for embedding training
    #do this for each example for each hero as hero_to_predict
    #if hero_to_predict<5, call it a radiant_hero, else dire
    #    then take hero to predict into target_df, and construct
    #    opp_side_heroes and same_side_heroes
    for hero_to_predict in range(10):
        same_side_hero_cols = []
        opp_side_hero_cols =[]
        if hero_to_predict<5:
            hero_list = list(range(5))
            hero_list.pop(hero_to_predict)
            hero_to_predict_list.extend(dota2_df['radiant_heroes'+str(hero_to_predict)])
            for hero in hero_list:
                same_side_hero_cols.append('radiant_heroes'+str(hero))
            for hero in range(5):
                opp_side_hero_cols.append('dire_heroes'+str(hero))
        else:
            hero_list = list(range(5))
            hero_list.pop(hero_to_predict-5)
            hero_to_predict_list.extend(dota2_df['dire_heroes'+str(hero_to_predict-5)])
            for hero in hero_list:
                same_side_hero_cols.append('dire_heroes'+str(hero))
            for hero in range(5):
                opp_side_hero_cols.append('radiant_heroes'+str(hero))
        same_side_heroes.extend(list(np.array(dota2_df.loc[:,same_side_hero_cols]).astype(int)))
        opp_side_heroes.extend(list(np.array(dota2_df.loc[:,opp_side_hero_cols]).astype(int)))
    feature_df['same_side_heroes'] = same_side_heroes
    feature_df['opp_side_heroes'] = opp_side_heroes
    target_df['hero_to_predict'] = hero_to_predict_list
    print(len(hero_to_predict_list))
    return feature_df,target_df

In [26]:
def preprocess_features(dota2_df):
    """Take dota2_df and create a dataframe containing only the features for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of heroes for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    radiant_heroes = list(np.array(dota2_df.loc[:,['radiant_heroes0',
                                                   'radiant_heroes1',
                                                   'radiant_heroes2',
                                                   'radiant_heroes3',
                                                   'radiant_heroes4']]).astype(int))
    processed_df['radiant_heroes'] = radiant_heroes
    
    dire_heroes = list(np.array(dota2_df.loc[:,['dire_heroes0',
                                                'dire_heroes1',
                                                'dire_heroes2',
                                                'dire_heroes3',
                                                'dire_heroes4']]).astype(int))        
    processed_df['dire_heroes'] = dire_heroes
     
    return processed_df

def preprocess_features_indivheroes(dota2_df):
    """Take dota2_df and create a dataframe containing only the features for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of heroes for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    #fiver heroes per team 
    for i in range(5):
        processed_df['radiant_hero'+str(i)]=dota2_df['radiant_hero'+str(i)]
        processed_df['dire_hero'+str(i)]=dota2_df['dire_hero'+str(i)]
    
    processed_df['skill_level']=dota2_df['skill_level']
    
    return processed_df
def preprocess_targets_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model.
      (radiant_win here)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    
    return target_df

def preprocess_targets_gold(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['gold_diff'] = (dota2_df['radiant_gold']-dota2_df['dire_gold'])\
                                /(dota2_df['radiant_gold']+dota2_df['dire_gold'])
    target_df['gold_diff'] = (target_df['gold_diff']-target_df['gold_diff'].mean())\
                                /target_df['gold_diff'].std()
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)

    return target_df

def z_score(column):
    return (column-column.mean())/column.std()

def preprocess_targets_dual_gold_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    return target_df

def preprocess_targets_triple_gold_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    target_df['total_gold'] = dota2_df['radiant_gold']+dota2_df['dire_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])
    target_df['total_gold'] = target_df['total_gold'].fillna(0)
    
    return target_df

def preprocess_targets_penta_gold_win_towdam(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    target_df['total_gold'] = dota2_df['radiant_gold']+dota2_df['dire_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])
    target_df['total_gold'] = target_df['total_gold'].fillna(0)
    
    target_df['towdam_diff'] = dota2_df['radiant_towerdamage']-dota2_df['dire_towerdamage']
    target_df['towdam_diff'] = z_score(target_df['towdam_diff'])
    target_df['towdam_diff'] = target_df['towdam_diff'].fillna(0)
    
    target_df['total_towdam'] = dota2_df['radiant_towerdamage']+dota2_df['dire_towerdamage']
    target_df['total_towdam'] = z_score(target_df['total_towdam'])
    target_df['total_towdam'] = target_df['total_towdam'].fillna(0)
    return target_df

def preprocess_targets_xp(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized xp difference in this case)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (xp_diff)
    """
    target_df = pd.DataFrame()
    target_df['xp_diff'] = (dota2_df['radiant_xp']-dota2_df['dire_xp'])/(dota2_df['radiant_xp']+dota2_df['dire_xp'])
    target_df['xp_diff'] = (target_df['xp_diff']-target_df['xp_diff'].mean())/target_df['xp_diff'].std()
    return target_df

### Visualize data for each set

In [17]:
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len

plotting_df = dota2_df

display.display(plotting_df.iloc[:train_len,:].describe())
display.display(plotting_df.iloc[train_len:train_len+test_len,:].describe())
display.display(plotting_df.iloc[train_len+test_len:,:].describe())


,Unnamed: 0,match_id,start_time,duration,game_mode,lobby_type,radiant_win,radiant_kills,dire_kills,radiant_gold,...,radiant_tower_status,dire_tower_status,first_blood_time,radiant_hero0,dire_hero0,radiant_hero1,dire_hero1,radiant_hero2,dire_hero2,radiant_hero3
count,201422.000000,2.014220e+05,2.014220e+05,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,...,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000,201422.000000
mean,143750.405278,4.757236e+09,1.558036e+09,1884.314499,20.428846,3.573085,0.547055,31.437033,30.610117,82700.252922,...,1085.672126,926.578452,94.582469,52.797371,52.803120,53.021457,53.105495,52.899907,53.105733,52.811873
std,83113.528557,1.987512e+07,5.787072e+05,689.063757,5.606372,3.499246,0.497782,13.370176,14.416025,32677.146247,...,924.554984,911.840420,77.887825,35.706884,35.661507,35.810210,35.800258,35.756952,35.746049,35.761002
min,0.000000,4.725403e+09,1.557130e+09,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71802.250000,4.733831e+09,1.557367e+09,1375.000000,22.000000,0.000000,0.000000,22.000000,20.000000,60842.025000,...,4.000000,4.000000,32.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,143528.500000,4.749398e+09,1.557800e+09,1875.000000,22.000000,7.000000,1.000000,31.000000,31.000000,80334.066667,...,1792.000000,390.000000,84.000000,47.000000,47.000000,48.000000,48.000000,47.000000,48.000000,47.000000
75%,215807.750000,4.775588e+09,1.558573e+09,2344.000000,23.000000,7.000000,1.000000,41.000000,41.000000,101868.791667,...,1975.000000,1972.000000,136.000000,83.000000,83.000000,84.000000,84.000000,84.000000,84.000000,84.000000
max,287745.000000,4.781085e+09,1.558752e+09,6168.000000,23.000000,7.000000,1.000000,115.000000,167.000000,444829.066667,...,2047.000000,2047.000000,879.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


,Unnamed: 0,match_id,start_time,duration,game_mode,lobby_type,radiant_win,radiant_kills,dire_kills,radiant_gold,...,radiant_tower_status,dire_tower_status,first_blood_time,radiant_hero0,dire_hero0,radiant_hero1,dire_hero1,radiant_hero2,dire_hero2,radiant_hero3
count,57550.000000,5.755000e+04,5.755000e+04,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,...,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000,57550.000000
mean,143794.858766,4.757184e+09,1.558034e+09,1882.227889,20.427315,3.546950,0.547507,31.352354,30.574318,82623.911158,...,1087.805943,926.354822,94.507472,52.661355,53.242259,52.978019,53.114996,53.144275,53.160382,53.197046
std,83062.733137,1.991099e+07,5.796787e+05,685.843580,5.605023,3.499715,0.497742,13.382557,14.363351,32558.558875,...,923.473366,911.841123,77.401974,35.610605,35.809650,35.762256,35.573579,35.720305,35.716632,35.725885
min,2.000000,4.725422e+09,1.557130e+09,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71889.000000,4.733829e+09,1.557367e+09,1373.000000,22.000000,0.000000,0.000000,22.000000,20.000000,60851.358333,...,4.000000,4.000000,32.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,144212.000000,4.749397e+09,1.557800e+09,1876.000000,22.000000,7.000000,1.000000,31.000000,31.000000,80457.100000,...,1792.000000,390.000000,84.000000,47.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
75%,215643.000000,4.775588e+09,1.558573e+09,2347.000000,23.000000,7.000000,1.000000,41.000000,41.000000,101795.229167,...,1975.000000,1972.000000,136.000000,83.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
max,287743.000000,4.781083e+09,1.558752e+09,7123.000000,23.000000,7.000000,1.000000,110.000000,167.000000,340716.833333,...,2047.000000,2047.000000,699.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


,Unnamed: 0,match_id,start_time,duration,game_mode,lobby_type,radiant_win,radiant_kills,dire_kills,radiant_gold,...,radiant_tower_status,dire_tower_status,first_blood_time,radiant_hero0,dire_hero0,radiant_hero1,dire_hero1,radiant_hero2,dire_hero2,radiant_hero3
count,28775.000000,2.877500e+04,2.877500e+04,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,...,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000,28775.000000
mean,144887.432737,4.757032e+09,1.558030e+09,1878.086603,20.418315,3.543423,0.549470,31.357915,30.411990,82583.366822,...,1090.109331,923.017376,94.019566,53.460608,52.812546,52.791208,53.114579,53.071242,52.867385,53.008862
std,82730.524592,1.991573e+07,5.798314e+05,690.735505,5.628454,3.499791,0.497555,13.351555,14.380586,32767.313728,...,925.475673,911.291485,77.356476,35.858234,35.608458,35.706991,35.749059,35.717571,35.872320,35.757728
min,13.000000,4.725406e+09,1.557130e+09,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,73219.500000,4.733827e+09,1.557367e+09,1374.000000,22.000000,0.000000,0.000000,22.000000,20.000000,60787.500000,...,4.000000,4.000000,31.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,145849.000000,4.749389e+09,1.557799e+09,1869.000000,22.000000,7.000000,1.000000,31.000000,30.000000,80294.250000,...,1792.000000,390.000000,84.000000,48.000000,47.000000,48.000000,48.000000,48.000000,47.000000,48.000000
75%,216194.500000,4.775587e+09,1.558573e+09,2336.000000,23.000000,7.000000,1.000000,41.000000,41.000000,101737.975000,...,1975.000000,1972.000000,136.000000,84.000000,83.000000,83.000000,84.000000,84.000000,84.000000,84.000000
max,287746.000000,4.781076e+09,1.558752e+09,5125.000000,23.000000,7.000000,1.000000,88.000000,112.000000,324866.500000,...,2047.000000,2047.000000,767.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


## Make tfrecords files

### Embedding training tfrecord making

In [10]:
#create tfrecord that needs a vocab for use in feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len


target_name='radiant_win'
train_features,train_targets = feature_target_hero_split(dota2_df.iloc[:train_len,:])
validation_features,validation_targets = feature_target_hero_split(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features,test_targets = feature_target_hero_split(dota2_df.iloc[train_len+validation_len:,:])


convert_to_tfrecords_embedding(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     'embedding_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords_embedding(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     'embedding_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_embedding(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     'embedding_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)


2011910
574830
287420
2011910
Data progress: 0/2011910
Data progress: 5000/2011910
Data progress: 10000/2011910
Data progress: 15000/2011910
Data progress: 20000/2011910
Data progress: 25000/2011910
Data progress: 30000/2011910
Data progress: 35000/2011910
Data progress: 40000/2011910
Data progress: 45000/2011910
Data progress: 50000/2011910
Data progress: 55000/2011910
Data progress: 60000/2011910
Data progress: 65000/2011910
Data progress: 70000/2011910
Data progress: 75000/2011910
Data progress: 80000/2011910
Data progress: 85000/2011910
Data progress: 90000/2011910
Data progress: 95000/2011910
Data progress: 100000/2011910
Data progress: 105000/2011910
Data progress: 110000/2011910
Data progress: 115000/2011910
Data progress: 120000/2011910
Data progress: 125000/2011910
Data progress: 130000/2011910
Data progress: 135000/2011910
Data progress: 140000/2011910
Data progress: 145000/2011910
Data progress: 150000/2011910
Data progress: 155000/2011910
Data progress: 160000/2011910
Data 

Data progress: 1355000/2011910
Data progress: 1360000/2011910
Data progress: 1365000/2011910
Data progress: 1370000/2011910
Data progress: 1375000/2011910
Data progress: 1380000/2011910
Data progress: 1385000/2011910
Data progress: 1390000/2011910
Data progress: 1395000/2011910
Data progress: 1400000/2011910
Data progress: 1405000/2011910
Data progress: 1410000/2011910
Data progress: 1415000/2011910
Data progress: 1420000/2011910
Data progress: 1425000/2011910
Data progress: 1430000/2011910
Data progress: 1435000/2011910
Data progress: 1440000/2011910
Data progress: 1445000/2011910
Data progress: 1450000/2011910
Data progress: 1455000/2011910
Data progress: 1460000/2011910
Data progress: 1465000/2011910
Data progress: 1470000/2011910
Data progress: 1475000/2011910
Data progress: 1480000/2011910
Data progress: 1485000/2011910
Data progress: 1490000/2011910
Data progress: 1495000/2011910
Data progress: 1500000/2011910
Data progress: 1505000/2011910
Data progress: 1510000/2011910
Data pro

Data progress: 425000/574830
Data progress: 430000/574830
Data progress: 435000/574830
Data progress: 440000/574830
Data progress: 445000/574830
Data progress: 450000/574830
Data progress: 455000/574830
Data progress: 460000/574830
Data progress: 465000/574830
Data progress: 470000/574830
Data progress: 475000/574830
Data progress: 480000/574830
Data progress: 485000/574830
Data progress: 490000/574830
Data progress: 495000/574830
Data progress: 500000/574830
Data progress: 505000/574830
Data progress: 510000/574830
Data progress: 515000/574830
Data progress: 520000/574830
Data progress: 525000/574830
Data progress: 530000/574830
Data progress: 535000/574830
Data progress: 540000/574830
Data progress: 545000/574830
Data progress: 550000/574830
Data progress: 555000/574830
Data progress: 560000/574830
Data progress: 565000/574830
Data progress: 570000/574830


### Single target, individual hero tfrecord making

In [58]:
#create tfrecord that needs a vocab for use in feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len


target_name='radiant_win_indivhero'
train_features = preprocess_features_indivheroes(dota2_df.iloc[:train_len,:])
test_features = preprocess_features_indivheroes(dota2_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features_indivheroes(dota2_df.iloc[train_len+test_len:,:])

train_targets = preprocess_targets_win(dota2_df.iloc[:train_len,:])
test_targets = preprocess_targets_win(dota2_df.iloc[train_len:train_len+test_len,:])
validation_targets = preprocess_targets_win(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_indivheroes(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords_indivheroes(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_indivheroes(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201422
Data progress: 5000/201422
Data progress: 10000/201422
Data progress: 15000/201422
Data progress: 20000/201422
Data progress: 25000/201422
Data progress: 30000/201422
Data progress: 35000/201422
Data progress: 40000/201422
Data progress: 45000/201422
Data progress: 50000/201422
Data progress: 55000/201422
Data progress: 60000/201422
Data progress: 65000/201422
Data progress: 70000/201422
Data progress: 75000/201422
Data progress: 80000/201422
Data progress: 85000/201422
Data progress: 90000/201422
Data progress: 95000/201422
Data progress: 100000/201422
Data progress: 105000/201422
Data progress: 110000/201422
Data progress: 115000/201422
Data progress: 120000/201422
Data progress: 125000/201422
Data progress: 130000/201422
Data progress: 135000/201422
Data progress: 140000/201422
Data progress: 145000/201422
Data progress: 150000/201422
Data progress: 155000/201422
Data progress: 160000/201422
Data progress: 165000/201422
Data progress: 170000/201422
Data progr

### Single target tfrecord making

In [18]:
print(train_targets.isnull().sum(),
      validation_targets.isnull().sum(),
      test_targets.isnull().sum())

NameError: name 'train_targets' is not defined

In [23]:
#create tfrecord that needs a vocab for use in feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len


target_name='radiant_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len+test_len:,:])

if target_name=='radiant_win':
    train_targets = preprocess_targets_win(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='gold_diff':
    train_targets = preprocess_targets_gold(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='xp_diff':
    train_targets = preprocess_targets_xp(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_xp(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_xp(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     target_name+'dota2_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     target_name+'dota2_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     target_name+'dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Data progress: 0/201191
Data progress: 5000/201191
Data progress: 10000/201191
Data progress: 15000/201191
Data progress: 20000/201191
Data progress: 25000/201191
Data progress: 30000/201191
Data progress: 35000/201191
Data progress: 40000/201191
Data progress: 45000/201191
Data progress: 50000/201191
Data progress: 55000/201191
Data progress: 60000/201191
Data progress: 65000/201191
Data progress: 70000/201191
Data progress: 75000/201191
Data progress: 80000/201191
Data progress: 85000/201191
Data progress: 90000/201191
Data progress: 95000/201191
Data progress: 100000/201191
Data progress: 105000/201191
Data progress: 110000/201191
Data progress: 115000/201191
Data progress: 120000/201191
Data progress: 125000/201191
Data progress: 130000/201191
Data progress: 135000/201191
Data progress: 140000/201191
Data progress: 145000/201191
Data progress: 150000/201191
Data progress: 155000/201191
Data progress: 160000/201191
Data progress: 165000/201191
Data progress: 170000/201191
Data progr

In [24]:
#create no vocab tfrecord for use in no feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len


target_name='radiant_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len+test_len:,:])

if target_name=='radiant_win':
    train_targets = preprocess_targets_win(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='gold_diff':
    train_targets = preprocess_targets_gold(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='xp_diff':
    train_targets = preprocess_targets_xp(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_xp(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_xp(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     target_name+'_novocab_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     target_name+'_novocab_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     target_name+'_novocab_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Data progress: 0/201191
Data progress: 5000/201191
Data progress: 10000/201191
Data progress: 15000/201191
Data progress: 20000/201191
Data progress: 25000/201191
Data progress: 30000/201191
Data progress: 35000/201191
Data progress: 40000/201191
Data progress: 45000/201191
Data progress: 50000/201191
Data progress: 55000/201191
Data progress: 60000/201191
Data progress: 65000/201191
Data progress: 70000/201191
Data progress: 75000/201191
Data progress: 80000/201191
Data progress: 85000/201191
Data progress: 90000/201191
Data progress: 95000/201191
Data progress: 100000/201191
Data progress: 105000/201191
Data progress: 110000/201191
Data progress: 115000/201191
Data progress: 120000/201191
Data progress: 125000/201191
Data progress: 130000/201191
Data progress: 135000/201191
Data progress: 140000/201191
Data progress: 145000/201191
Data progress: 150000/201191
Data progress: 155000/201191
Data progress: 160000/201191
Data progress: 165000/201191
Data progress: 170000/201191
Data progr

### Make dual gold_win tfrecord files

In [21]:
#save tfrecord file with vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='dual_gold_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_dual_gold_win(\
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


In [40]:
#save dual gold win tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='dual_gold_win_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_dual_gold_win(\
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


### Save triple target gold win tfrecord with no vocab

In [47]:
#save triple gold win tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='triple_gold_win_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_triple_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_triple_gold_win(
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_triple_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


### Save penta target gold, win, tower_damage tfrecord

In [35]:
#save penta gold win tower damage tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='penta_gold_win_towdam_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_penta_gold_win_towdam(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_penta_gold_win_towdam(
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_penta_gold_win_towdam(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0
